In [1]:
import numpy as np
from functools import reduce
import math
import copy
from PIL import Image

In [2]:
def resi_sistem(a, b, c, d):
    a = np.array([[a[0], b[0], c[0]],
                  [a[1], b[1], c[1]],
                  [a[2], b[2], c[2]]])
    b = np.array([d[0], d[1], d[2]])
    return np.linalg.solve(a, b)

In [3]:
a = [-3, -1, 1]
b = [3, -1, 1]
c = [1, 1, 1]
d = [-1, 1, 1]
ap = [-2, -1, 1]
bp = [2, -1, 1]
cp = [2, 1, 1]
dp = [-2, 1, 1]
tacke = [a, b, c, d]
tacke_p = [ap, bp, cp, dp]

In [4]:
resi_sistem(a, b, c, d)

array([ 0.33333333, -0.33333333,  1.        ])

In [5]:
def naivni_algoritam(tacke, tacke_p):
    a, b, c, d = tacke
    ap, bp, cp, dp = tacke_p
    
    #dobijamo parametre resavanjem sistema jednacina 
    #tako da je D = alpha * A + beta * B + gama * D
    params = resi_sistem(a, b, c, d)
    alfa, beta, gama = params
    
    #matrica sa kolonama alfa * A, beta * B, gama * C
    P1 = np.array([[alfa*a[0], beta*b[0], gama*c[0]],
                   [alfa*a[1], beta*b[1], gama*c[1]],
                   [alfa *a[2], beta*b[2], gama*c[2]]])
    
    #isti postupak i za tacke koje predstavljaju slike
    params_p = resi_sistem(ap, bp, cp, dp)
    alfa_p, beta_p, gama_p = params_p
    
    P2 = np.array([[alfa_p*ap[0], beta_p*bp[0], gama_p*cp[0]],
                   [alfa_p*ap[1], beta_p*bp[1], gama_p*cp[1]],
                   [alfa_p*ap[2], beta_p*bp[2], gama_p*cp[2]]])
    
    #matrica preslikavanja P = P2 * P1 ^ -1
    P = np.dot(P2, np.linalg.inv(P1))
    return P

In [6]:
a = [-3, -1, 1]
b = [3, -1, 1]
c = [1, 1, 1]
d = [-1, 1, 1]
ap = [-2, -1, 1]
bp = [2, -1, 1]
cp = [2, 1, 1]
dp = [-2, 1, 1]
tacke = [a, b, c, d]
tacke_p = [ap, bp, cp, dp]

In [7]:
P = naivni_algoritam(tacke, tacke_p)
print('Naivni algoritam:')
print(P)
print('Zaokruzeno na 5 decimala:')
print(np.around(P, 5))

Naivni algoritam:
[[ 2.00000000e+00 -6.66133815e-16  6.66133815e-16]
 [-4.44089210e-16  2.00000000e+00 -1.00000000e+00]
 [ 4.44089210e-16 -1.00000000e+00  2.00000000e+00]]
Zaokruzeno na 5 decimala:
[[ 2. -0.  0.]
 [-0.  2. -1.]
 [ 0. -1.  2.]]


In [8]:
def napravi_matricu_2x9(t, tp):
    return np.matrix(
        [[0, 0, 0, -tp[2]*t[0], -tp[2]*t[1], -tp[2]*t[2], tp[1]*t[0], tp[1]*t[1], tp[1]*t[2]],
         [tp[2]*t[0], tp[2]*t[1], tp[2]*t[2], 0, 0, 0, -tp[0]*t[0], -tp[0]*t[1], -tp[0]*t[2]]])

In [9]:
def dlt_algoritam(tacke, tacke_p):
    n = len(tacke)
    
    #za svaku tacku originala i njenu sliku odredjujemo 2x9 matricu 
    #i spajamo te matrice u jednu 2nx9 matricu
    A = napravi_matricu_2x9(tacke[0], tacke_p[0])
    
    for i in range(1, n):
        matrica = napravi_matricu_2x9(tacke[i], tacke_p[i])
        A = np.concatenate((A, matrica), axis = 0)
    
    #odredjujemo SVD dekompoziciju matrice A, A = U*D*V^T, koja minimizuje 
    #odredjenu statisticku ili geometrijsku gresku
    U, D, V = np.linalg.svd(A)

    #trazeno preslikavanje je poslednja kolona matrice V
    V = np.transpose(V)
    P = V[:,-1].reshape(3, 3)

    return np.array(P)

In [10]:
#provaeravamo za 4 korespodencije
Pnaive = naivni_algoritam(tacke, tacke_p)
Pdlt = dlt_algoritam(tacke, tacke_p)
print('Naivni algoritam:')
print(Pnaive)
print('Naivni zaokruzen na 5 decimala:')
print(np.around(Pnaive, 5))
print('Dlt algoritam:')
print(Pdlt)
print('Dlt zaokruzena na 5 decimala:')
print(np.around(Pdlt, 5))
print('Poredimo da li dobijamo proporcionalne matrice:')
print(np.around(Pdlt / Pdlt[1, 1] * Pnaive[1, 1], 5))

Naivni algoritam:
[[ 2.00000000e+00 -6.66133815e-16  6.66133815e-16]
 [-4.44089210e-16  2.00000000e+00 -1.00000000e+00]
 [ 4.44089210e-16 -1.00000000e+00  2.00000000e+00]]
Naivni zaokruzen na 5 decimala:
[[ 2. -0.  0.]
 [-0.  2. -1.]
 [ 0. -1.  2.]]
Dlt algoritam:
[[ 5.34522484e-01  6.18413875e-17 -2.58635702e-16]
 [-5.55111512e-17  5.34522484e-01 -2.67261242e-01]
 [-5.55111512e-17 -2.67261242e-01  5.34522484e-01]]
Dlt zaokruzena na 5 decimala:
[[ 0.53452  0.      -0.     ]
 [-0.       0.53452 -0.26726]
 [-0.      -0.26726  0.53452]]
Poredimo da li dobijamo proporcionalne matrice:
[[ 2.  0. -0.]
 [-0.  2. -1.]
 [-0. -1.  2.]]


In [11]:
Pnaive = naivni_algoritam(tacke, tacke_p)
#proveravamo za 6 korespodencija
e = [1, 2, 3]
f = [-8, -2, 1]
ep = [2, 1, 4]
fp = [-16, -5, 4]
tacke = [a, b, c, d, e, f]
tacke_p = [ap, bp, cp, dp, ep, fp]
P = dlt_algoritam(tacke, tacke_p)
print('Matrica dobijena DLT algoritmom:')
print(P)
print('Dlt zaokruzen na 5 decimala:')
print(np.around(P, 5))
#proveravamo da li su matrice dobijene dlt algoritmom
#i naivnim algoritmom proprocionalne
#odnosno da li zadaju isto projektivno preslikavanje
print('Da li dobijamo proporcionalne matrice:')
print(np.around(P / P[1, 1] * Pnaive[1, 1], 5))

Matrica dobijena DLT algoritmom:
[[-5.34522484e-01  1.41553436e-15 -1.12323345e-15]
 [ 3.60822483e-16 -5.34522484e-01  2.67261242e-01]
 [-2.35922393e-16  2.67261242e-01 -5.34522484e-01]]
Dlt zaokruzen na 5 decimala:
[[-0.53452  0.      -0.     ]
 [ 0.      -0.53452  0.26726]
 [-0.       0.26726 -0.53452]]
Da li dobijamo proporcionalne matrice:
[[ 2. -0.  0.]
 [-0.  2. -1.]
 [ 0. -1.  2.]]


In [12]:
def normalizacija(tacke):
    n = len(tacke)
    
    #izracunavamo afino teziste sistema
    tx = 0
    ty = 0
    for i in range(n):
        tx += float(tacke[i][0]/tacke[i][2])
        ty += float(tacke[i][1]/tacke[i][2])
        
    teziste = np.array([tx, ty])
    teziste = teziste * np.array([1/n, 1/n])
    
    #dobijamo matricu translacije tako sto
    #transliramo teziste u koordinatni pocetak
    G = np.matrix([
        [1, 0, -teziste[0]],
        [0, 1, -teziste[1]],
        [0, 0, 1]
    ])
    
    #skaliranje tacaka tako da prosecna udaljenost
    #od koordinatnog pocetka bude sqrt(2)
    distance = 0
    for i in range(n):
        distance += math.sqrt((tacke[i][0]/tacke[i][2] - teziste[0])**2+(tacke[i][1]/tacke[i][2] - teziste[1])**2)    
    distance /= n
    
    k = np.sqrt(2) / distance * 1.0
    
    #dobijamo matricu homotetije S kojom se skaliraju tacke
    S = np.matrix([[k, 0, 0],
                   [0, k, 0],
                   [0, 0, 1]])
    
    #matrica normalizacije T = S*G
    T = np.dot(S, G)
    
    return np.array(T)

In [13]:
print(normalizacija(tacke))

[[0.4496856  0.         0.57459826]
 [0.         0.4496856  0.09993013]
 [0.         0.         1.        ]]


In [14]:
def normalizovani_dlt(tacke, tacke_p):
    n = len(tacke)
    
    #normalizujemo tacke originale i njihove slike
    T = normalizacija(tacke)
    Tp = normalizacija(tacke_p)
 
    #primenimo matricu normalizacije na svaku tacku T*tacke[i]
    nove_tacke = copy.deepcopy(tacke)
    nove_tacke_p = copy.deepcopy(tacke_p)
    for i in range(n):
        [x, y, z] = np.dot(T, [tacke[i][0], tacke[i][1], tacke[i][2]]) 
        nove_tacke[i][0] = x
        nove_tacke[i][1] = y
        nove_tacke[i][2] = z

    #isto uradimo i za slike Tp*tacke_p[i]
    for i in range(n):
        [x, y, z] = np.dot(Tp, [tacke_p[i][0], tacke_p[i][1], tacke_p[i][2]]) 
        nove_tacke_p[i][0] = x
        nove_tacke_p[i][1] = y
        nove_tacke_p[i][2] = z
        
    #odredjujemo matricu Pp dlt algoritmom primenjenim na normalizovane tacke
    Pp = dlt_algoritam(nove_tacke, nove_tacke_p)
    
    #matrica transformacije P = Tp^-1 * P * T
    P = np.dot(np.dot(np.linalg.inv(Tp), Pp), T)
    
    return P

In [15]:
D = normalizovani_dlt(tacke, tacke_p)
print('Normalizovani Dlt:')
print(D)
print('Normalizovani dlt zaokruzen na 5 decimala:')
print(np.around(D, 5))
print('Da li je preslikavanje proporcionalno:')
print(np.around(D / D[1, 1] * Pnaive[1, 1]))

Normalizovani Dlt:
[[-3.97002511e-01 -3.72227716e-16  1.11022302e-16]
 [-4.10609963e-17 -3.97002511e-01  1.98501255e-01]
 [ 1.24812826e-16  1.98501255e-01 -3.97002511e-01]]
Normalizovani dlt zaokruzen na 5 decimala:
[[-0.397  -0.      0.    ]
 [-0.     -0.397   0.1985]
 [ 0.      0.1985 -0.397 ]]
Da li je preslikavanje proporcionalno:
[[ 2.  0. -0.]
 [ 0.  2. -1.]
 [-0. -1.  2.]]


In [16]:
def poredjenje():
    #poredjenje Naivnog i DLT algoritma sa 4 korespodencije
    a = [1, 1, 1]
    b = [5, 2, 1]
    c = [6, 4, 1]
    d = [-1, 7, 1]
    ap = [0, 0, 1]
    bp = [10, 0, 1]
    cp = [10, 5, 1]
    dp = [0, 5, 1]
    tacke = np.array([a, b, c, d])
    tacke_p =  np.array([ap, bp, cp, dp])
    
    Anaive = naivni_algoritam(tacke, tacke_p)
    Adlt = dlt_algoritam(tacke, tacke_p)
    
    print('POREDIMO NAIVNI I DLT')
    print('Naivni algoritam:')
    print(Anaive)
    print('Dlt algoritam:')
    print(Adlt)
    print('Da li su matrice proporcionalne:')
    print(Adlt / Adlt[1, 1] * Anaive[1, 1])
    print()
    
    #dodajemo 5. tacku
    e = [3, 1, 1]
    ep = [3, -1, 1]
    tacke = np.array([a, b, c, d, e])
    tacke_p =  np.array([ap, bp, cp, dp, ep])
    
    Adlt5 = dlt_algoritam(tacke, tacke_p)
    Bdlt5 = dlt_algoritam([a, c, b, d, e], [ap, cp, bp, dp, ep])
    
    print('OSOBINE DLT ALGORITMA')
    print('Dlt sa 4 tacke:')
    print(Adlt)
    print('Dlt sa 5 tacaka:')
    print(Adlt5)
    print('Da li je osetljiv na permutaciju odredjenih tacaka (menjamo b i c):')
    print(Bdlt5)
    print()
    
    #da li je DLT invarijantan na promenu koordinata
    #kada izaberemo razlicite koordinate istih tacaka
    #npr stavimo koordinatni pocetak u centar slike umesto u gornji levi ugao
    
    #matrica transformacije koordinata originala
    T = np.array([[0, 1, 2], [-1, 0, 3], [0, 0, 1]])
    #matrica transformacije koordinata slika
    Tp = np.array([[1, -1, 5], [1, 1, -2], [0, 0, 1]])

    #dobijamo nove koordinate originala i slika
    #primenom trasformacija T i Tp na stare tacke
    nove_tacke = []
    nove_tacke_p = []
    for i in range(len(tacke)):
        nove_tacke.append(np.dot(T, tacke[i]))
        nove_tacke_p.append(np.dot(Tp, tacke_p[i]))
    
    P_stare = dlt_algoritam(tacke, tacke_p)
    P_nove = dlt_algoritam(nove_tacke, nove_tacke_p)
    
    #stari dlt u novim koordinatama Tp^-1 * P_nove * T
    P_tmp = np.dot(np.dot(np.linalg.inv(Tp), P_nove), T)
    
    print("DA LI JE DLT INVARIJANTAN NA PROMENU KOORDINATA")
    print("Dlt matrica starih tacaka")
    print(P_stare)
    print("Dlt matrica novih tacaka:")
    print(P_nove)
    print("Matrica starih tacaka u novim koordinatama: ")
    print(P_tmp)
    P_tmp = (P_tmp/P_tmp[0][0])*P_stare[0][0]
    #pojavljuje se razlika na drugoj decimali, nije invarijantan
    print('Reskaliramo matrice:')
    print(P_tmp)
    print()
    
    #poredimo modifikovani dlt i dlt
    Adlt = dlt_algoritam(tacke, tacke_p)
    AdltNorm = normalizovani_dlt(tacke, tacke_p)
    P_stare = normalizovani_dlt(tacke, tacke_p)
    P_nove = normalizovani_dlt(nove_tacke, nove_tacke_p)
    print("POREDIMO MODIFIKOVANI DLT I DLT:")
    print("Dlt algoritam: ")
    print(Adlt)
    print("Normalizovani dlt algoritam:")
    print(AdltNorm)
    #skaliramo da uporedimo rzultate
    print('Skalirano:')
    print(Adlt / Adlt[1][1] * AdltNorm[1][1])
    print()

    # proveravamo da li je rezultat modifikovanog dlt algoritma 
    # prrimenjenog na nove koordinate isti kao rezultat
    # starog u novim koordinatama
    
    #vracamo u stari koordinatni sistem
    P_tmp= np.dot(np.dot(np.linalg.inv(Tp), P_nove), T)
    print("DA LI MODIFIKOVANI DLT ZAVISI OD IZBORA KOORDINATA")
    print('Normalizovani dlt sa starim koordinatama:')
    print(P_stare)
    print("Normalizovani dlt u novim koordinatama:")
    print(P_nove)
    print("Normalizovani dlt starih tacaka u novim koordinatama: ")
    print(P_tmp)
    print("Skaliramo da uporedimo:")
    print(P_stare / P_stare[1][1] * P_tmp[1][1])


In [17]:
poredjenje()

POREDIMO NAIVNI I DLT
Naivni algoritam:
[[ 0.44871795  0.14957265 -0.5982906 ]
 [-0.19230769  0.76923077 -0.57692308]
 [-0.07277526  0.07378079  0.41075918]]
Dlt algoritam:
[[ 0.3418787   0.11395957 -0.45583826]
 [-0.14651944  0.58607777 -0.43955833]
 [-0.05544755  0.05621367  0.31295787]]
Da li su matrice proporcionalne:
[[ 0.44871795  0.14957265 -0.5982906 ]
 [-0.19230769  0.76923077 -0.57692308]
 [-0.07277526  0.07378079  0.41075918]]

OSOBINE DLT ALGORITMA
Dlt sa 4 tacke:
[[ 0.3418787   0.11395957 -0.45583826]
 [-0.14651944  0.58607777 -0.43955833]
 [-0.05544755  0.05621367  0.31295787]]
Dlt sa 5 tacaka:
[[ 0.34349761  0.11823681 -0.48379032]
 [-0.15246704  0.58506938 -0.39503374]
 [-0.05732138  0.05556327  0.32557538]]
Da li je osetljiv na permutaciju odredjenih tacaka (menjamo b i c):
[[-0.34349761 -0.11823681  0.48379032]
 [ 0.15246704 -0.58506938  0.39503374]
 [ 0.05732138 -0.05556327 -0.32557538]]

DA LI JE DLT INVARIJANTAN NA PROMENU KOORDINATA
Dlt matrica starih tacaka
[[ 0.

In [18]:
def unos_koordinata(n):
    koordinate = np.zeros((n, 3))
    for i in range(n):
        koordinate[i][0] = int(input("Unesite x koordinatu:"))
        koordinate[i][1] = int(input("Unesite y koordinatu:"))
        koordinate[i][2] = int(input("Unesite z koordinatu:"))
        
    return koordinate

In [48]:
def ispravljanje_distorzije():
    slika = Image.open('tower.jpg')
    slika.show()
    dimenzije = slika.size
    #pravimo crnu sliku 
    nova_slika = Image.new('RGB', dimenzije, 'black')

    #dobijamo stare i nove pixele
    stari_pixel = slika.load()
    novi_pixel = nova_slika.load()
    
    #korisnik bira algoritam
    tip_algoritma = input("Koji algoritam zelite da koristite? (dlt/naivni)")

    #korisnik unosi koordinate
    tacke = unos_koordinata(4)

    A = tacke[0]
    B = tacke[1]
    C = tacke[2]
    D = tacke[3]

    AD = math.sqrt((A[0]-D[0])**2 +(A[1]-D[1])**2)
    BC = math.sqrt((B[0]-C[0])**2 +(B[1]-C[1])**2)

    AB = math.sqrt((A[0]-B[0])**2 +(A[1]-B[1])**2)
    DC = math.sqrt((D[0]-C[0])**2 +(D[1]-C[1])**2)

    vertikalna_sredina = (AD + BC)/2
    horizontalna_sredina = (AB + DC)/2

    # koordinate slika odredjujemo na osnovu prosecne duzine vertikalnih i horizontalnih duzi
    # gornja leva tacka ostaje ista, a ostale se racunaju tako da obrazuju pravougaonik 
    # i da su udaljene od te tacke za prosecnu duzinu
    tacke_p = [[A[0],A[1],1],
               [A[0]+horizontalna_sredina, A[1], 1], 
               [A[0]+horizontalna_sredina,A[1]+vertikalna_sredina, 1], 
               [A[0], A[1]+vertikalna_sredina,1]]

    if(tip_algoritma == "dlt"):
        P = dlt_algoritam(tacke, tacke_p)
        P = np.linalg.inv(P)
    else:
        P = naivni_algoritam(tacke, tacke_p)
        P = np.linalg.inv(P)

    #za svaku koordinatu nove slike odredjujemo koordinate originalne slike i 
    #vrednost pixela na novoj slici je jednak vrednosti pixela koji se nalazi
    #na koordinatama stare slike
    for i in range(dimenzije[0]):
        for j in range(dimenzije[1]):
            nove = np.dot(P, [i,j,1])

            x = round(nove[0] / nove[2])
            y = round(nove[1] / nove[2])

            if (x<0 or x>=dimenzije[0]) or (y<0 or y>=dimenzije[1]):
                continue
            else:
                novi_pixel[i,j] = stari_pixel[x,y]
            
    nova_slika.save("slika.bmp")
    nova_slika.show()

In [49]:
ispravljanje_distorzije()

Koji algoritam zelite da koristite? (dlt/naivni)dlt
Unesite x koordinatu:200
Unesite y koordinatu:150
Unesite z koordinatu:1
Unesite x koordinatu:550
Unesite y koordinatu:65
Unesite z koordinatu:1
Unesite x koordinatu:550
Unesite y koordinatu:450
Unesite z koordinatu:1
Unesite x koordinatu:200
Unesite y koordinatu:400
Unesite z koordinatu:1


In [21]:
a=[-3,2,1]
b=[-2,5,2]
c=[1,0,3]
d=[-7,3,1]
e=[2,1,2]
f=[-1,2,1]
g=[1,1,1]

In [22]:
ap = [11,-12,7]
bp = [25,-8,9]
cp = [15,4,17]
dp = [14,-28,10]
ep=[13,8,9]
fp=[11,-4,5]
gp=[8.02,4,4]
originali = [a, b, c, d, e, f, g]
slike = [ap, bp, cp, dp, ep, fp, gp]

In [47]:
np.around(naivni_algoritam([a, b, c, d], [ap, bp, cp, dp]), 5)
naivni = naivni_algoritam([a, b, c, d], [ap, bp, cp, dp])
naivni

array([[ 1.23430677e-15,  3.00000000e+00,  5.00000000e+00],
       [ 4.00000000e+00, -2.08983158e-16, -4.01639506e-16],
       [-1.00000000e+00, -1.00000000e+00,  6.00000000e+00]])

In [24]:
dlt = dlt_algoritam(originali, slike)
dlt

array([[ 5.30528511e-05,  3.20011319e-01,  5.32929220e-01],
       [ 4.26445525e-01, -2.02187024e-05, -2.95145391e-05],
       [-1.06595323e-01, -1.06519060e-01,  6.39542445e-01]])

In [25]:
m = dlt / dlt[0, 1] * 3

In [26]:
m

array([[ 4.97352886e-04,  3.00000000e+00,  4.99603472e+00],
       [ 3.99778538e+00, -1.89543630e-04, -2.76689017e-04],
       [-9.99295809e-01, -9.98580867e-01,  5.99549835e+00]])

In [27]:
np.around(m, 2)

array([[ 0.,  3.,  5.],
       [ 4., -0., -0.],
       [-1., -1.,  6.]])

In [28]:
ndlt = normalizovani_dlt(originali, slike)
ndlt

array([[ 3.89785465e-05,  2.34697334e-01,  3.90851376e-01],
       [ 3.12756187e-01, -1.48613871e-05, -2.16907546e-05],
       [-7.81772458e-02, -7.81212009e-02,  4.69041798e-01]])

In [29]:
nm = ndlt / ndlt[0, 1] * 3

In [30]:
nm

array([[ 4.98240170e-04,  3.00000000e+00,  4.99602662e+00],
       [ 3.99778107e+00, -1.89964499e-04, -2.77260345e-04],
       [-9.99294424e-01, -9.98578036e-01,  5.99548946e+00]])

In [31]:
cc=[[0,1,2],[-1,0,3],[0,0,1]]

In [32]:
norig = []
nslike = []
for i in range(len(originali)):
    norig.append(np.dot(cc, originali[i]))
    nslike.append(np.dot(cc, slike[i]))

In [33]:
m1 = dlt_algoritam(norig, nslike)

In [34]:
m1s = np.dot(np.dot(np.linalg.inv(cc), m1), cc)

In [35]:
m1s

array([[-1.73564676e-05, -1.04847554e-01, -1.74607666e-01],
       [-1.39719503e-01,  6.61760914e-06,  9.63372330e-06],
       [ 3.49246285e-02,  3.48997143e-02, -2.09538156e-01]])

In [36]:
mm = m1s / m1s[0, 1] * 3

In [37]:
mm

array([[ 4.96620099e-04,  3.00000000e+00,  4.99604404e+00],
       [ 3.99779004e+00, -1.89349457e-04, -2.75649443e-04],
       [-9.99297373e-01, -9.98584504e-01,  5.99550913e+00]])

In [38]:
m

array([[ 4.97352886e-04,  3.00000000e+00,  4.99603472e+00],
       [ 3.99778538e+00, -1.89543630e-04, -2.76689017e-04],
       [-9.99295809e-01, -9.98580867e-01,  5.99549835e+00]])

In [39]:
m2 = normalizovani_dlt(norig, nslike)

In [40]:
m2s = np.dot(np.dot(np.linalg.inv(cc), m2), cc)
m2s

array([[ 0.16005754, -0.20193015, -0.06045275],
       [-0.43635812, -0.24554245,  0.21376057],
       [ 0.12937776,  0.02404026, -0.1657667 ]])

In [41]:
nm2 = m2s / m2s[0, 1] * 3
nm2

array([[-2.37791436,  3.        ,  0.89812363],
       [ 6.48280767,  3.64793139, -3.17575996],
       [-1.92211656, -0.35715713,  2.46273322]])

In [42]:
nm

array([[ 4.98240170e-04,  3.00000000e+00,  4.99602662e+00],
       [ 3.99778107e+00, -1.89964499e-04, -2.77260345e-04],
       [-9.99294424e-01, -9.98578036e-01,  5.99548946e+00]])

In [43]:
def test_algoritama(originali, slike):
    originali_naivni = originali[:4]
    slike_naivni = slike[:4]
    print('Naivni algoritam:')
    print(naivni_algoritam(originali_naivni, slike_naivni))
    print()
    print('Dlt algoritam:')
    dlt = dlt_algoritam(originali, slike)
    print(dlt)
    print()
    ndlt = normalizovani_dlt(originali, slike)
    print('Modifikovani dlt:')
    print(ndlt)

In [44]:
test_algoritama(originali, slike)

Naivni algoritam:
[[ 1.23430677e-15  3.00000000e+00  5.00000000e+00]
 [ 4.00000000e+00 -2.08983158e-16 -4.01639506e-16]
 [-1.00000000e+00 -1.00000000e+00  6.00000000e+00]]

Dlt algoritam:
[[ 5.30528511e-05  3.20011319e-01  5.32929220e-01]
 [ 4.26445525e-01 -2.02187024e-05 -2.95145391e-05]
 [-1.06595323e-01 -1.06519060e-01  6.39542445e-01]]

Modifikovani dlt:
[[ 3.89785465e-05  2.34697334e-01  3.90851376e-01]
 [ 3.12756187e-01 -1.48613871e-05 -2.16907546e-05]
 [-7.81772458e-02 -7.81212009e-02  4.69041798e-01]]
